In [1]:
import numpy as np
import os
import pandas as pd
import string as str
import math
import sys
import time
import scipy.sparse as spr

from scipy import optimize, special
import gurobipy as grb

from sklearn.preprocessing import LabelEncoder
from IPython.display import display, HTML

In [2]:
def two_d(X):
    return np.reshape(X,(X.size, 1))

In [3]:
thepath = os.path.join(os.getcwd(),'..')
travelmode = pd.read_csv(os.path.join(thepath,'data_mec_optim/demand_travelmode/travelmodedata.csv'))


In [4]:
lb = LabelEncoder() 
travelmode['choice'] = lb.fit_transform(travelmode['choice'])

In [5]:
nobs = travelmode.shape[0]
ncols = travelmode.shape[1]
nbchoices = 4
ninds = int(nobs/nbchoices)

In [6]:
muhat_i_y = travelmode['choice'].values.reshape(ninds,nbchoices).T
muhat_iy = muhat_i_y.flatten()

In [7]:
sorter = ['air', 'train', 'bus', 'car']
travelmode['mode'] = travelmode['mode'].astype("category")
travelmode['mode'].cat.set_categories(sorter, inplace=True)
travelmode.columns = travelmode.columns.str.strip()
travelmode.sort_values(['mode','individual'], inplace = True)

In [8]:
travelmode.head()

,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1,air,0,69,59,100,70,35,1
4,2,air,0,64,58,68,68,30,2
8,3,air,0,69,115,125,129,40,1
12,4,air,0,64,49,68,59,70,3
16,5,air,0,64,60,144,82,45,2


In [9]:
Phi_iy_k = np.column_stack((np.kron(np.identity(4)[0:4,1:4],np.repeat(1, ninds).reshape(ninds,1)), - travelmode['travel'].values, - (travelmode['travel']*travelmode['income']).values, - travelmode['gcost'].values))

In [10]:
nbK = Phi_iy_k.shape[1]
phi_mean = Phi_iy_k.mean(axis = 0)
phi_stdev = Phi_iy_k.std(axis = 0, ddof = 1)
Phi_iy_k = ((Phi_iy_k - phi_mean).T/phi_stdev[:,None]).T

In [11]:
def log_likelihood(theta):
    nbK = np.asarray(theta).shape[0]
    Xtheta = Phi_iy_k.dot(theta)/sigma
    Xthetamat_iy = Xtheta.reshape(nbchoices, ninds).T
    max_i = np.amax(Xthetamat_iy, axis = 1)
    expPhi_iy = np.exp((Xthetamat_iy.T -max_i).T)
    d_i = np.sum(expPhi_iy, axis = 1)
    
    val = np.sum(np.multiply(Xtheta,muhat_iy))  - np.sum(max_i) - sigma * np.sum(np.log(d_i))

    return -val

In [12]:
def grad_log_likelihood(theta):
    nbK = np.asarray(theta).shape[0]
    Xtheta = Phi_iy_k.dot(theta)/sigma
    Xthetamat_iy = Xtheta.reshape(nbchoices, ninds).T
    max_i = np.amax(Xthetamat_iy, axis = 1)
    expPhi_iy = np.exp((Xthetamat_iy.T -max_i).T)
    d_i = np.sum(expPhi_iy, axis = 1)
    
    temp_mat = np.multiply(Phi_iy_k.T, expPhi_iy.T.flatten()).T
    list_temp = []
    for i in range(nbchoices):
        list_temp.append(temp_mat[i*ninds:(i+1)*ninds,])
    n_i_k = np.sum(list_temp,axis = 0)
    
    thegrad = muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten() - np.sum(n_i_k.T/d_i, axis = 1)

    return -thegrad

In [13]:
theta0 = np.repeat(0,nbK)
sigma = 1
outcome = optimize.minimize(log_likelihood,method = 'CG',jac = grad_log_likelihood, x0 = theta0)

In [14]:
outcome

     fun: 264.6629616781814
     jac: array([-1.99967610e-06, -1.23457774e-07,  1.10935096e-06,  9.27663557e-07,
        7.06908573e-07,  6.59686510e-07])
 message: 'Optimization terminated successfully.'
    nfev: 59
     nit: 31
    njev: 59
  status: 0
 success: True
       x: array([0.71120459, 0.36631111, 0.50859505, 0.41287867, 0.52396689,
       0.43816127])

In [15]:
temp_mle = 1 / outcome['x'][nbK - 1]
theta_mle = outcome['x']*temp_mle
print(temp_mle)
print(theta_mle)

2.282264698256094
[1.62315714 0.83601891 1.16074852 0.94229841 1.19583112 1.        ]


In [16]:
lenobj = nbK+ninds
c = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1,ninds)))

m = grb.Model('lp')
x = m.addMVar(lenobj, name='x', lb=-grb.GRB.INFINITY)
m.setObjective(c @ x, grb.GRB.MAXIMIZE)
cstMat = spr.hstack((spr.csr_matrix(-Phi_iy_k), spr.kron(two_d(np.repeat(1,nbchoices)),spr.identity(ninds))))
rhs = np.repeat(0,ninds*nbchoices)
m.addConstr(cstMat @ x >= rhs)
nbCstr = cstMat.shape[0]
const_2 = np.array([0]*(nbK - 1))
const_2 = np.append(const_2, 1)
const_2 = np.append(const_2 ,[0]*ninds)
m.addConstr(const_2 @ x == 1)
m.optimize()
if m.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", m.objval)
    opt_x = m.getAttr('x')

Using license file C:\Users\jmcgn\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 841 rows, 216 columns and 5881 nonzeros
Model fingerprint: 0xd18b21ff
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 211 rows and 1 columns
Presolve time: 0.01s
Presolved: 630 rows, 215 columns, 2520 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     340   -5.2644494e+01   0.000000e+00   0.000000e+00      0s

Solved in 340 iterations and 0.03 seconds
Optimal objective -5.264449397e+01
Value of the problem (Gurobi) = -52.644493974371535


In [17]:
theta_lp = np.array(opt_x[:nbK])
print(theta_lp)
print(theta_mle)

[-0.12239648 -0.24955822 -0.32097987 -0.48746045  0.00598981  1.        ]
[1.62315714 0.83601891 1.16074852 0.94229841 1.19583112 1.        ]


In [18]:
indMax=100
tempMax=temp_mle
outcomemat = np.zeros((indMax+1,nbK-1))

In [19]:
def log_likelihood_fixedtemp(subsetoftheta, *temp):
    val = log_likelihood(np.append(subsetoftheta, 1/temp[0]))
    
    return val

In [20]:
def grad_log_likelihood_fixedtemp(subsetoftheta, *temp):
    val = grad_log_likelihood(np.append(subsetoftheta, 1/temp[0]))[:-1]
    
    return val

In [21]:
outcomemat[0,:] = theta_lp[:-1]
iterMax = indMax+1
for k in range(2,iterMax+1,1):
    thetemp = tempMax * (k-1)/indMax
    outcomeFixedTemp = optimize.minimize(log_likelihood_fixedtemp,method = 'CG',jac = grad_log_likelihood_fixedtemp, args = (thetemp,),  x0 = theta0[:-1])
    outcomemat[k-1,:] = outcomeFixedTemp['x']*thetemp

In [22]:
outcomemat

array([[-1.22396477e-01, -2.49558217e-01, -3.20979871e-01,
        -4.87460448e-01,  5.98981140e-03],
       [-1.26219650e-01, -2.52231709e-01, -3.25554733e-01,
        -4.73489491e-01, -1.15577050e-02],
       [-1.24567917e-01, -2.50040402e-01, -3.26033829e-01,
        -4.62939452e-01, -1.94887622e-02],
       [-1.18157369e-01, -2.46825031e-01, -3.22964293e-01,
        -4.53609530e-01, -2.07100446e-02],
       [-1.09873666e-01, -2.44497524e-01, -3.19065236e-01,
        -4.44867552e-01, -1.96914464e-02],
       [-1.00127575e-01, -2.42036379e-01, -3.14326570e-01,
        -4.36207041e-01, -1.70577711e-02],
       [-8.91486020e-02, -2.38854108e-01, -3.08602366e-01,
        -4.27263177e-01, -1.31280975e-02],
       [-7.71418357e-02, -2.34766725e-01, -3.01842313e-01,
        -4.17831706e-01, -8.12097332e-03],
       [-6.42814532e-02, -2.29782088e-01, -2.94068510e-01,
        -4.07836228e-01, -2.17790501e-03],
       [-5.07077831e-02, -2.23979969e-01, -2.85344405e-01,
        -3.97278287e-01

The zero-temperature estimator is:

In [23]:
print(outcomemat[1,:])

[-0.12621965 -0.25223171 -0.32555473 -0.47348949 -0.01155771]


The mle estimator is:

In [24]:
print(outcomemat[indMax,])

[1.62315732 0.83601911 1.1607487  0.94229764 1.19583231]


In [51]:
nbB = 100
thetemp = 1
epsilon_biy = special.digamma(1) -np.log(-np.log(np.random.uniform(0,1,ninds*nbchoices*nbB)))
lenobj = ninds*nbB+nbK

newc = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1/nbB,ninds*nbB)))
newm = grb.Model('new_lp')
x = newm.addMVar(lenobj, name='x', lb=-grb.GRB.INFINITY)
newm.setObjective(newc @ x, grb.GRB.MAXIMIZE)
mat1 = spr.kron(-Phi_iy_k, two_d(np.repeat(1,nbB)))
mat2 = spr.kron(two_d(np.repeat(1,nbchoices)),spr.identity(ninds*nbB))
newcstMat = spr.hstack((mat1, mat2))
rhs = epsilon_biy
newm.addConstr(newcstMat @ x >= rhs)
newm.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 84000 rows, 21006 columns and 588000 nonzeros
Model fingerprint: 0x27b7367c
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-05, 1e+01]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.36s
Presolved: 21006 rows, 84000 columns, 588000 nonzeros

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 1.260e+05
 Factor NZ  : 1.474e+05 (roughly 40 MBytes of memory)
 Factor Ops : 1.041e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.65319293e+02  6.80320933e-01  2.75e+00 8.20e+00  1.24e+01     1s
   1   4.82307115e+00 -3.76287065e+03  2.26e-02 1.42e-14  1.46e-01     1s
   2  -2.12323620e+01 -1.64855716e+03  9.91e-04 1.42e-14  2.1

In [85]:
if m.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", newm.objval)
    opt_x = np.array(newm.getAttr('x'))
newtheta_lp = opt_x[:nbK] / opt_x[nbK-1]

Value of the problem (Gurobi) = -265.7684409315817


In [86]:
print(theta_mle)
print(newtheta_lp)

[1.62315714 0.83601891 1.16074852 0.94229841 1.19583112 1.        ]
[1.60694702 0.81865359 1.14556472 0.9146398  1.22612242 1.        ]


Finally probit

In [88]:
nbB = 100
thetemp = 1
epsilon_biy = np.random.normal(nbB*ninds*nbchoices)
lenobj = ninds*nbB+nbK

newc = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1/nbB,ninds*nbB)))
newm = grb.Model('new_lp')
x = newm.addMVar(lenobj, name='x', lb=-grb.GRB.INFINITY)
newm.setObjective(newc @ x, grb.GRB.MAXIMIZE)
mat1 = spr.kron(-Phi_iy_k, two_d(np.repeat(1,nbB)))
mat2 = spr.kron(two_d(np.repeat(1,nbchoices)),spr.identity(ninds*nbB))
newcstMat = spr.hstack((mat1, mat2))
rhs = epsilon_biy
newm.addConstr(newcstMat @ x >= rhs)
newm.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 84000 rows, 21006 columns and 588000 nonzeros
Model fingerprint: 0xfea60542
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+04, 8e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.26s
Presolved: 21006 rows, 84000 columns, 588000 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.275e+03
 Factor NZ  : 1.869e+03
 Factor Ops : 2.260e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.69270900e+08 -1.76401283e+07  2.29e+00 9.31e-10  5.37e+00     0s
   1  -2.35297072e+07 -1.76403768e+07  5.36e-02 2.62e-10  3.99e-01     0s
   2  -1.76401283e+07 -1.76401419e+07  3.77e-15 3.13e-10  1.62e-02     0s
   3  -1.76401283